# WHAT IS THIS NOTEBOOK ABOUT?

This notebook contains the different efforts to obtain reliable weather information. If any forecasting is tested, it may be tested here as well

### Contents
1. [OpenMeteo](#OPEN_METEO)

# IMPORTS

In [30]:
import requests
import pandas as pd
import maya
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

# OPEN_METEO

In [84]:
def ManageDateTime(ds_obj):
    return datetime.strftime(maya.parse(ds_obj).datetime(), "%Y-%m-%d %H:%M:%S")

class InvalidArgument(Exception):
    "Input Either City name or Lat AND Lon for city"
    pass
def GetWeatherData(city = "None", lon = "no", lat= "no", start_date= "yesterday", end_date= "today"):
    if city != "None":
        url = "https://geocoding-api.open-meteo.com/v1/search?name={city_name}&count=10&language=en&format=json".format(city_name = city)
        geo_ = requests.get(url)
        results_ = geo_.json()["results"][0]
        lat = results_["latitude"]
        lon = results_["longitude"]
    elif lat == "no" or lon == "no":
        raise InvalidArgument
    start_date_parsed = datetime.strftime(maya.when(start_date).datetime(), "%Y-%m-%d")
    end_date_parsed = datetime.strftime(maya.when(end_date).datetime(), "%Y-%m-%d")
    url = "https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={start_date}&end_date={end_date}&hourly=temperature_2m,cloudcover,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance"\
        .format(lat = lat, lon = lon, start_date = start_date_parsed, end_date = end_date_parsed)
    data = requests.get(url)
    try:
        data = data.json()["hourly"]
    except:
        print(data)
    data = pd.DataFrame(data)
    data["ds"] = data["time"].apply(ManageDateTime)

    url = "https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&past_days=10&hourly=temperature_2m,cloudcover,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance"\
        .format(latitude = lat, longitude = lon)
    forecast_data = requests.get(url)
    forecast_data = pd.DataFrame(forecast_data.json()["hourly"])
    forecast_data["ds"] = forecast_data["time"].apply(ManageDateTime)

    data["type"] = "real"
    forecast_data["type"] = "forecast"


    return pd.concat([data, forecast_data], ignore_index= True)


In [91]:
data = GetWeatherData("Ghent", start_date= "1 Apr 2023")

In [92]:
data

,time,temperature_2m,cloudcover,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,ds,type
0,2023-04-01T00:00,9.0,100.0,0.0,0.0,0.0,0.0,2023-04-01 00:00:00,real
1,2023-04-01T01:00,8.8,100.0,0.0,0.0,0.0,0.0,2023-04-01 01:00:00,real
2,2023-04-01T02:00,8.7,100.0,0.0,0.0,0.0,0.0,2023-04-01 02:00:00,real
3,2023-04-01T03:00,8.4,100.0,0.0,0.0,0.0,0.0,2023-04-01 03:00:00,real
4,2023-04-01T04:00,8.3,100.0,0.0,0.0,0.0,0.0,2023-04-01 04:00:00,real
...,...,...,...,...,...,...,...,...,...
979,2023-04-30T19:00,14.5,80.0,19.0,0.0,19.0,2.8,2023-04-30 19:00:00,forecast
980,2023-04-30T20:00,13.2,61.0,0.0,0.0,0.0,0.0,2023-04-30 20:00:00,forecast
981,2023-04-30T21:00,11.7,41.0,0.0,0.0,0.0,0.0,2023-04-30 21:00:00,forecast
982,2023-04-30T22:00,11.1,61.0,0.0,0.0,0.0,0.0,2023-04-30 22:00:00,forecast


In [93]:
fig = px.scatter(data, x = "ds", y = "temperature_2m", color = "type")
fig.update_layout(
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Poppins',
                size=12,
                color='rgb(117,161,51)',
            )
        ),
        yaxis = dict(
            showgrid = False,
            linewidth = 2,
            ticks = "outside",
            tickfont = dict(
                family = "Poppins",
                size = 14,
                color = 'rgb(117,161,51)'
            ),
            showticklabels = True
        ),
        showlegend=True,
        title = "Temperature Time Series",
        plot_bgcolor='rgb(255,255,255)',
        xaxis_title='Datetime',
        yaxis_title='Temperature (ºC)',
        height = 350
    )
fig.show()

In [94]:
fig = px.scatter(data, x = "ds", y = "direct_radiation", color = "type")
fig.update_layout(
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Poppins',
                size=12,
                color='rgb(117,161,51)',
            )
        ),
        yaxis = dict(
            showgrid = False,
            linewidth = 2,
            ticks = "outside",
            tickfont = dict(
                family = "Poppins",
                size = 14,
                color = 'rgb(117,161,51)'
            ),
            showticklabels = True
        ),
        showlegend=True,
        title = "Direct Radiation Series",
        plot_bgcolor='rgb(255,255,255)',
        xaxis_title='Datetime',
        yaxis_title='Direct Radiation',
        height = 350
    )
fig.show()